In [2]:
import pandas as pd

### Code to feature selection and  transform the raw crime data file 

In [58]:
import os 
currentDirectory = os.getcwd()

crimeDataRaw  = pd.read_csv(f'{currentDirectory}/data/bocsarCrimeData.csv')

crimeDataRaw  = crimeDataRaw[['Postcode','Offence category','Subcategory','Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021',
       'Jul 2021', 'Aug 2021', 'Sep 2021', 'Oct 2021', 'Nov 2021', 'Dec 2021']]

crimeDataRaw['TotalCrimes'] =  crimeDataRaw[['Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021',
       'Jul 2021', 'Aug 2021', 'Sep 2021', 'Oct 2021', 'Nov 2021', 'Dec 2021']].sum( axis=1)

crimeDataRaw.to_csv(f'{currentDirectory}/data/crime_data_transformed.csv',index=False)


,Postcode,Offence category,Subcategory,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021,Jun 2021,Jul 2021,Aug 2021,Sep 2021,Oct 2021,Nov 2021,Dec 2021,TotalCrimes
0,2000,Homicide,Murder *,0,0,0,1,0,1,0,0,0,0,0,0,2
1,2000,Homicide,Attempted murder,0,1,0,0,0,0,0,0,0,0,0,0,1
2,2000,Homicide,"Murder accessory, conspiracy",0,0,0,0,0,0,0,0,0,0,0,0,0
3,2000,Homicide,Manslaughter *,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2000,Assault,Domestic violence related assault,20,28,14,27,20,14,12,10,11,21,28,26,231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39117,4385,Against justice procedures,Fail to appear,0,0,0,0,0,0,0,0,0,0,0,0,0
39118,4385,Against justice procedures,Resist or hinder officer,0,0,0,0,0,0,0,0,0,0,0,0,0
39119,4385,Against justice procedures,Other offences against justice procedures,0,0,0,0,0,0,0,0,0,0,0,0,0
39120,4385,Transport regulatory offences,Transport regulatory offences,0,0,0,0,0,0,0,0,0,0,0,0,0


## Code to merge two datasets

In [ ]:
crimeData = pd.read_csv(f'{currentDirectory}/data/crime_data_transformed.csv' )

In [23]:
censusData    = pd.read_csv('/Users/icmec/Downloads/2021_GCP_POA_for_NSW_short-header/2021 Census GCP Postal Areas for NSW/2021Census_G01_NSW_POA.csv')

def remove_poa_string(x):
    return str(x)[3:]

censusData['POA_CODE_2021'] =  censusData['POA_CODE_2021'].apply( remove_poa_string )  #will i need to filter by children or by people

censusData.rename( columns = {'POA_CODE_2021':'Postcode'} , inplace=True)
censusData['Postcode'] = censusData['Postcode'].astype('int64')


In [79]:
censusData.shape

(612, 109)

In [27]:
crimeData[ crimeData['Subcategory'] == 'Domestic violence related assault'] 
(crimeData['Postcode'].unique())

#crimeData[crimeData['Postcode'] == 2003]

,Postcode,Offence category,Subcategory,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021,Jun 2021,Jul 2021,Aug 2021,Sep 2021,Oct 2021,Nov 2021,Dec 2021,TotalCrimes
4,2000,Assault,Domestic violence related assault,20,28,14,27,20,14,12,10,11,21,28,26,231
66,2006,Assault,Domestic violence related assault,0,0,0,0,0,0,0,0,0,0,0,0,0
128,2007,Assault,Domestic violence related assault,3,1,1,3,3,1,3,2,5,3,3,3,31
190,2008,Assault,Domestic violence related assault,1,1,2,4,6,8,2,0,3,1,1,4,33
252,2009,Assault,Domestic violence related assault,5,3,6,12,3,8,3,1,3,3,2,5,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38816,4375,Assault,Domestic violence related assault,0,0,0,0,0,0,0,0,0,0,0,0,0
38878,4377,Assault,Domestic violence related assault,0,0,0,0,0,0,0,0,0,0,0,0,0
38940,4380,Assault,Domestic violence related assault,0,0,0,0,0,0,0,0,1,0,0,0,1
39002,4383,Assault,Domestic violence related assault,1,0,0,1,0,0,0,0,0,0,0,0,2


In [14]:
def filterSubcategory( df , subcategory ):  
    return df[ df.Subcategory == subcategory ]

In [43]:
filteredSubCategory = filterSubcategory(crimeData,'Domestic violence related assault')

In [58]:

merged = pd.merge(filteredSubCategory,censusData[['Postcode','Tot_P_P']],on='Postcode')

In [59]:
merged['rate'] = merged['TotalCrimes']/merged['Tot_P_P']

In [61]:
merged.to_csv('domesticViolence.csv')

In [67]:
filteredSubCategory = filterSubcategory(crimeData,'Sexual touching, sexual act and other sexual offences')
merged = pd.merge(filteredSubCategory,censusData[['Postcode','Tot_P_P']],on='Postcode')
merged['rate'] = merged['TotalCrimes']/merged['Tot_P_P']
merged.to_csv('SexualOffence.csv')


In [68]:
filteredSubCategory = filterSubcategory(crimeData,'Abduction and kidnapping')
merged = pd.merge(filteredSubCategory,censusData[['Postcode','Tot_P_P']],on='Postcode')
merged['rate'] = merged['TotalCrimes']/merged['Tot_P_P']
merged.to_csv('Abduction_and_kidnapping.csv')

In [85]:
offencesRelatedCSE = crimeData[ crimeData.Subcategory.isin(['Sexual assault','Blackmail and extortion', 'Intimidation, stalking and harassment','Pornography offences','Prostitution offences']) ] 
merged = pd.merge(offencesRelatedCSE,censusData[['Postcode','Tot_P_P']],on='Postcode')
merged['rate'] = merged['TotalCrimes']/merged['Tot_P_P']
merged.to_csv('offencesRelatedCSE.csv')

In [87]:
merged = pd.merge(offencesRelatedCSE,censusData[['Postcode','Tot_P_P']],on='Postcode')
merged['rate'] = merged['TotalCrimes']/merged['Tot_P_P']
merged.to_csv('offencesRelatedCSE.csv')

# Exploring postcode that are absent in Census data but in Crime date 

In [ ]:
#crime recorded in postcodes not in census 
postcodesNotInCensus = list(set(crimeData.Postcode) - set(censusData.Postcode))

crimeData[  (crimeData['Subcategory'] == 'Sexual assault') &  (crimeData['TotalCrimes'] > 0) & (crimeData['Postcode'].isin(postcodesNotInCensus))  ]


In [101]:
some = crimeData[    (crimeData['TotalCrimes'] > 0) & (crimeData['Postcode'].isin(postcodesNotInCensus))  ]


In [116]:
#number of postcodes that have no record in the census dataset ( the postcodes belong to two or more POA )
some['Postcode'].unique()

array([2006, 2052, 2109, 2123, 2129, 2139, 2314, 2351, 2522, 2611, 2618,
       2755, 3644, 3691, 3707, 4377, 4380, 4383, 4385])

In [82]:
crimeData[  (crimeData['Subcategory'] == 'Sexual assault') &  (crimeData['TotalCrimes'] > 0) & (crimeData['Postcode'].isin(postcodesNotInCensus))  ]

,Postcode,Offence category,Subcategory,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021,Jun 2021,Jul 2021,Aug 2021,Sep 2021,Oct 2021,Nov 2021,Dec 2021,TotalCrimes
69,2006,Sexual offences,Sexual assault,0,0,2,0,1,2,0,0,0,0,2,0,7
2673,2052,Sexual offences,Sexual assault,0,0,0,0,0,1,0,0,1,0,0,0,2
17181,2351,Sexual offences,Sexual assault,0,1,0,0,0,0,0,0,0,0,0,1,2
23815,2522,Sexual offences,Sexual assault,0,2,0,0,0,0,0,0,0,0,0,0,2
38633,3644,Sexual offences,Sexual assault,1,0,0,0,0,0,0,0,0,0,0,0,1


# Pushing the Bocsor data to Postgres 

In [1]:


import configparser
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import glob
import os 

parser = configparser.ConfigParser()
parser.read("/Users/icmec/Documents/CONFIGURATIONS/postgres.conf")
dbname = parser.get("postgres_config", "database")
user = parser.get("postgres_config", "username")
password = parser.get("postgres_config",
    "password")
host = parser.get("postgres_config", "host")
port = parser.get("postgres_config", "port")

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')
Session = sessionmaker(bind=engine)

crime_Data   = pd.read_csv('/Users/icmec/Documents/Project_census_crime/CrimeProcessing/data/bocsarCrimeData.csv')


with Session() as session:
    crime_Data.to_sql('bocsor.crimestats', con=engine, schema = 'init' , if_exists='replace',index=False )




# Pushing crimeRate for various offences to prod in postgres. 

In [100]:
import configparser
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, false
from sqlalchemy.orm import sessionmaker
import glob
import os 

parser = configparser.ConfigParser()
parser.read("/Users/icmec/Documents/CONFIGURATIONS/postgres.conf")
dbname = parser.get("postgres_config", "database")
user = parser.get("postgres_config", "username")
password = parser.get("postgres_config",
    "password")
host = parser.get("postgres_config", "host")
port = parser.get("postgres_config", "port")

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')
Session = sessionmaker(bind=engine)

crime_rate   = pd.read_csv('/Users/icmec/Documents/Project_census_crime/CrimeProcessing/data/offencesRelatedCSE.csv')

crime_rate = crime_rate[['Postcode','Offence category','Subcategory','TotalCrimes','Tot_P_P','rate']]
with Session() as session:
    crime_rate.to_sql('bocsar.crimestats.rate', con=engine, schema = 'prod' , if_exists='append',index=False )

In [98]:
from operator import index


crime_rate   = pd.read_csv('/Users/icmec/Documents/Project_census_crime/CrimeProcessing/data/offencesRelatedCSE.csv')

,Postcode,Offence category,Subcategory,TotalCrimes,Tot_P_P,rate
0,2000,Sexual offences,Sexual assault,70,27936,0.002506
1,2000,Blackmail and extortion,Blackmail and extortion,4,27936,0.000143
2,2000,"Intimidation, stalking and harassment","Intimidation, stalking and harassment",393,27936,0.014068
3,2000,Pornography offences,Pornography offences,3,27936,0.000107
4,2000,Prostitution offences,Prostitution offences,1,27936,0.000036
...,...,...,...,...,...,...
3050,2898,Sexual offences,Sexual assault,0,445,0.000000
3051,2898,Blackmail and extortion,Blackmail and extortion,0,445,0.000000
3052,2898,"Intimidation, stalking and harassment","Intimidation, stalking and harassment",3,445,0.006742
3053,2898,Pornography offences,Pornography offences,0,445,0.000000


# this code is to upload the dictionary to postgres 

In [47]:
from helper import *  
# import helper
# import importlib
# importlib.reload(helper)

pao_dictionary_df = pd.read_excel('/Users/icmec/Downloads/2021_GCP_POA_for_NSW_short-header/Metadata/Metadata_2021_GCP_DataPacks_R1_R2.xlsx' , skiprows=10 , sheet_name='Cell Descriptors Information' )

postgresConf = GetConfiguration()
SendData(pao_dictionary_df,table_name='2021_census.gcp_poa_nsw.dict',  if_exists_flag = 'replace'  )


'success'